In [1]:
import cymetric as cym
import pandas as pd
#import sklearn as skl
import math

import numpy as np
import random
import matplotlib.pyplot as plt
import graphviz
plt.rcParams['figure.figsize'] = (40,24)
from cymetric import graphs as cgr
from cymetric import timeseries as tm
from cymetric import filters as fl

In [2]:
file = "small_time_step/run1.sqlite"
db = cym.dbopen(file)
ev = cym.Evaluator(db=db, write=True)
#cym.graphs.flow_graph(ev, label = "mass")

#### extract data from the sqlite file

In [3]:
def tidy_df(file):
    """
    Extracts agents, transactions, and resource data from cyclus output file 
    and returns a pandas dataframe of transactions labeled with the sending and receiving facilities 
    and quantity of material exchanged
    """
    db = cym.dbopen(file)
    ev = cym.Evaluator(db=db, write=True)
    
    agentTable = ev.eval("AgentEntry")
    try: 
        agents = agentTable.loc[:, ["AgentId", "Prototype"]]
    except: 
        print("there was an agent issue with " + file)
    
    transTable = ev.eval("Transactions")
    try: 
        transactions = transTable.loc[:, ["SenderId", "ReceiverId", "ResourceId", "Commodity", "Time"]]
    except: 
        print("there was an transactions issue with " + file)
    
    resourceTable = ev.eval("Resources")
    try: 
        resources = resourceTable.loc[:, ["ResourceId", "Quantity"]]
    except:
        print("there was a resources issue with " + file)
    
    #merge agents, transactions, and resources
    int1 = pd.merge(transactions, resources, on='ResourceId', how='inner')

    #rename AgentId column to facilitate merge 
    send = agents.rename(columns = {"AgentId": "SenderId"})
    receive = agents.rename(columns = {"AgentId": "ReceiverId"})

    for i in range(len(int1)):
        for j in range(len(send)):
            if int1.loc[i,"SenderId"] == send.loc[j, "SenderId"]:
                int1.loc[i,"SenderId"] = send.loc[j, "Prototype"]
                
    for i in range(len(int1)):
        for j in range(len(receive)):
            if int1.loc[i,"ReceiverId"] == receive.loc[j, "ReceiverId"]:
                int1.loc[i,"ReceiverId"] = receive.loc[j, "Prototype"]

                
#     print(send)
#     print(int1)
    
    return int1
    

In [4]:
tidy_df("small_time_step/run0.sqlite")

,SenderId,ReceiverId,ResourceId,Commodity,Time,Quantity
0,UraniumMine,LEUEnrichmentFacility,9,c_uore,0,1.025256e+06
1,UraniumMine,LEUEnrichmentFacility,21,c_uore,1,1.025256e+06
2,LEUEnrichmentFacility,LWR,27,c_leu,1,1.950000e+02
3,LEUEnrichmentFacility,LWR,33,c_leu,1,1.950000e+02
4,LEUEnrichmentFacility,LWR,39,c_leu,1,1.950000e+02
5,UraniumMine,LEUEnrichmentFacility,45,c_uore,2,1.025256e+06
6,LEUEnrichmentFacility,SpentFuelSink,48,c_spent_leu,2,1.560474e+03
7,LEUEnrichmentFacility,SpentFuelSink,49,c_spent_leu,2,1.560474e+03
8,LEUEnrichmentFacility,SpentFuelSink,38,c_spent_leu,2,1.560474e+03
9,UraniumMine,LEUEnrichmentFacility,55,c_uore,3,1.025256e+06


#### trim resource-identifying columns

In [5]:
def trim_data(df):
    """
    Returns a dataframe with extraneous columns from the tidy dataframe removed and creates two empty columns
    for the truck signal: fraction, which is the amount of material in the receiving facility relative to 
    the capacity of the truck; and truck, which is the integer number of trucks sent at that time step
    """
    trimmed = df[["SenderId", "ReceiverId", "Time", "Quantity"]]
    trimmed["fraction"] = pd.Series(0, index = range(365)) #316 what is this?? 
    trimmed["truck"] = pd.Series(0, index = range(365))
    
    #assume leu and heu enrichment happens in the same physical facility
    #change the name of enrichment facilites 
    trimmed["SenderId"] = trimmed['SenderId'].replace({'LEUenrich': 'enrichment', 'HEUenrich': 'enrichment'})
    trimmed['ReceiverId'] = trimmed['ReceiverId'].replace({'LEUenrich': 'enrichment', 'HEUenrich': 'enrichment'})
    
    return trimmed

#### define truck

for each transaction, cap shipment at a certain mass m ( = 25000kg? is realisitc, but a smaller value might be more interesting)
create a new column with the number of trucks sent for this interaction (0 + )
consider storing heu for several timesteps


In [6]:
from collections import Counter
def send_trucks(truck_df, truck_size): 
    """
    Returns a dataframe with the 'fraction' and 'truck' columns filled in 
    """
    truck_df["fraction"] = truck_df["Quantity"] / truck_size 
    
    #collect set of all transaction types: 
    transaction_pairs = Counter()
    for i in range(len(truck_df)): 
        #add this transaction to the counter
        #check if the value for that type of transaction is greater than the value for the truck 
        #if so, send a truck and update the value for the transaction type
        #if not, update the value for the transaction type 
        #0: senderid, 1: receiverid, 2: time, 3: quantity, 4: fraction, 5: truck
        sender = truck_df.iloc[i, 0]
        receiver = truck_df.iloc[i, 1]
        transaction_pairs.update({(sender, receiver): truck_df.iloc[i, 3]})
        stored_material = transaction_pairs[(sender, receiver)]
        if stored_material >= truck_size: 
            trucks = stored_material // truck_size
            truck_df.at[i, "truck"] = trucks
            transaction_pairs[(sender, receiver)] -= trucks * truck_size
        else: 
            truck_df.at[i, "truck"] = 0
        
            
#     print(transaction_pairs)
#     print("truck df")
#     print(truck_df)
        
    return truck_df
  

#### flatten data into single row for use with other runs

columns: every perumutation of transaction (truck boolean), every timestamp

row: single cyclus run 

transactions: mine to enrichment, enrichment to reactor, reactor to sf sink, enrichment to heu sink

#### modify this to look for its own facilities

### Isotope Signal

benchmark ratios from this paper: https://drive.google.com/file/d/14XZ71E9zuG6nXFM-v3a7aYGpraha3pD8/view

new "belch" happens after each reactor cycle end

In [7]:
def isotope_signal(filename): #parameters: duration, dt, some transaction table (check for reactor refueling)
    """
    Returns a dataframe with a six columns for each time step representing one of six xenon isotope ratios
    stretched across one row. The isotope signal is charactarized by an initial release of xenon at the end 
    of each reactor cycle accumulating throughout the simulation. 
    """
    #converted to seconds:
    half131m = (11.9*24*60*60) #days
    half133 = (5.25*24*60*60) #days
    half133m = (2.19*24*60*60) #days
    half135 = (9.10*60*60) #hours
    time_step = (30*24*60*60)
    
    l131m = -np.log(2) / half131m
    l133 = -np.log(2) / half133
    l133m = -np.log(2) / half133m
    l135 = -np.log(2) / half135
    
    tidydf = tidy_df(filename)

    #find all of the reactor cycle starts 
    #calculate ratios for each t in cycle
    isotope_rows = []
    '''
    for time step in time steps: 
        if before first delivery: 
            add only background (start with 0)
            
        calculate new ratios from previous time step
        
        if end of reactor cycle: 
            create new initial signal (n for each of the 4 isotopes)
            calculate ratios
            add to variable
        
        append ratios to row 
    '''
    duration = int(tidydf["Time"].max())
    #print("duration", duration)
    cycle_ends = tidydf[tidydf["SenderId"] == "LWR"]["Time"].to_list()
    #print(cycle_ends)
    
    for i, t in enumerate(range(366)): #duration+1)): 
        row = {"135/133m": 0, "135/133": 0, "135/131m": 0, \
               "133m/133": 0, "133m/131m": 0, "133/131m": 0}
        
        if cycle_ends and t >= cycle_ends[0]: #every time the reactor sends material elsewhere
            previous_row = isotope_rows[i-1]
            #Rn/m(t) = Rn/m(0)e^-(Ln - Lm)t
            row["135/133m"] = previous_row["135/133m"] * np.exp(-(l135-l133m)*t)
            row["135/133"] = previous_row["135/133"] * np.exp(-(l135-l133)*t)
            row["135/131m"] = previous_row["135/131m"] * np.exp(-(l135-l131m)*t)
            row["133m/133"] = previous_row["133m/133"] * np.exp(-(l133m-l133)*t)
            row["133m/131m"] = previous_row["133m/131m"] * np.exp(-(l133m-l131m)*t)
            row["133/131m"] = previous_row["133/131m"] * np.exp(-(l133-l131m)*t)
    
            if t in cycle_ends: #first isotopes released after first cycle
                #multiply each by random variable
                row["135/133m"]  += 607
                row["135/133"]   += 66.4
                row["135/131m"]  += 220000
                row["133m/133"]  += 0.109
                row["133m/131m"] += 363
                row["133/131m"]  += 3320

        isotope_rows.append(row)  
        
    isotope_columns = []
    
    for t in range(365): # no signal after last transaction
        for key, value in isotope_rows[t].items(): 
            isotope_columns.append({f'{key}_t{t}': value})

    isotope_df = pd.DataFrame(isotope_columns)
    long_row = isotope_df.sum().to_frame().T
    
    return long_row


In [8]:
#isotope_signal("small_time_step/run0.sqlite")

In [9]:
def make_cols(max_time):
    """
    Returns a list of column names: an initial column to label the presence of heu diversion and 
    a set of all possible transaction facility pairs for each time step
    """
    col_names = ["diversion"]
    #edit so that transactions include all but the heu going to the heusink 
    transactions = {("UraniumMine", "enrichment"), ("enrichment", "reactor"), 
                    ("reactor", "SpentFuelSink"), }#("enrichment", "HEUSink")}
    
    for t in range(max_time):
        for trans in sorted(transactions): 
            col_names.append(trans[0] + "--" + trans[1] + "|time" + str(t))
    return col_names
 
def make_row(filename, truckdf, max_time): 
    """
    Returns a list of values that will populate the columns created above in a single row representing 
    an individual cyclus run
    """
    long_row = []
    
    db = cym.dbopen(filename)
    ev = cym.Evaluator(db=db, write=True)
    
#     print(ev.eval("AgentEntry")["Prototype"].to_list())
#     print("HEUenrich" in ev.eval("AgentEntry")["Prototype"].to_list())
    
    long_row.append("HEUenrich" in ev.eval("AgentEntry")["Prototype"].to_list())

    transactions = {
        ("UraniumMine", "enrichment"), ("enrichment", "reactor"), 
                    ("reactor", "SpentFuelSink"), #("enrichment", "HEUSink")
    }
    sorted_trans = sorted(transactions)
    for t in range(max_time):
        #subset rows with this timestep
        subset = truckdf.loc[truckdf['Time'] == t]
        sub_row = [0] * len(transactions)
        
        for index, row in subset.iterrows():
            #check each possible transaction
            for t in range(len(transactions)): 
                if row["SenderId"] == sorted_trans[t][0] and row["ReceiverId"] == sorted_trans[t][1]:    
                    sub_row[t] = truckdf.loc[index, "truck"]

        long_row.extend(sub_row)
    return long_row

In [10]:
def file_to_line(filename, truck_size, max_time):
    """
    Returns a list (row) of data extracted from an indiviudal cyclus output file to be appended to the 
    simulation dataframe
    """
    return make_row(filename, send_trucks(trim_data(tidy_df(filename)), truck_size), max_time)
    
def simulation_data(files, truck_size, max_time):
    """
    Returns a wide dataframe of one row per cyclus run with columns for truck deliveries and xenon signals
    at each time step 
    """
    columns = make_cols(max_time)
    print("column length", len(columns))
    rows = []
    
    for file in files: 
        truck_columns = file_to_line(file, truck_size, max_time)
        print("number of truck cells in row", len(truck_columns))
        isotopes = isotope_signal(file)
        isotope_columns = isotopes.iloc[0].values.tolist()
        print("number of isotope cells in row", len(isotope_columns))
        rows.append(truck_columns + isotope_columns)
        if file == files[0]: 
#             print(isotopes.columns.to_list())
            columns.extend(isotopes.columns.to_list())
            print("after append col length", len(columns))
            
    return pd.DataFrame(data = rows, columns = columns)


In [11]:
simulation_data(["small_time_step/run0.sqlite", "small_time_step/run1.sqlite"], 12000, 365)
#simulation_data(["small_time_step/run0.sqlite"], 12000, 365)
#print(file_to_line("small_time_step/run0.sqlite", 1200, 365))

column length 1096


<ipython-input-5-3bf58e632af7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed["fraction"] = pd.Series(0, index = range(365)) #316 what is this??


number of truck cells in row 1096
number of isotope cells in row 2190
after append col length 3286


<ipython-input-5-3bf58e632af7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed["fraction"] = pd.Series(0, index = range(365)) #316 what is this??


number of truck cells in row 1096
number of isotope cells in row 2190


,diversion,UraniumMine--enrichment|time0,enrichment--reactor|time0,reactor--SpentFuelSink|time0,UraniumMine--enrichment|time1,enrichment--reactor|time1,reactor--SpentFuelSink|time1,UraniumMine--enrichment|time2,enrichment--reactor|time2,reactor--SpentFuelSink|time2,...,135/131m_t363,133m/133_t363,133m/131m_t363,133/131m_t363,135/133m_t364,135/133_t364,135/131m_t364,133m/133_t364,133m/131m_t364,133/131m_t364
0,False,0,0,0,0,0,0,0,0,0,...,2.879362e+06,0.598893,2072.001477,17234.889314,6895.949083,838.863883,2.900911e+06,0.599359,2074.257106,17240.247335
1,False,0,0,0,0,0,0,0,0,0,...,3.340674e+06,0.715004,2468.951329,20638.520101,8025.309506,974.061998,3.365676e+06,0.715560,2471.639087,20644.936251


In [12]:
import os
files = []
for file in os.listdir("small_time_step"):
    if file.endswith(".sqlite"):
        files.append(os.path.join("small_time_step", file))

print(files[0])
big_df = simulation_data(files[0:], 12000, 365)

small_time_step/run0.sqlite
column length 1096


<ipython-input-5-3bf58e632af7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed["fraction"] = pd.Series(0, index = range(365)) #316 what is this??


number of truck cells in row 1096
number of isotope cells in row 2190
after append col length 3286


<ipython-input-5-3bf58e632af7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed["fraction"] = pd.Series(0, index = range(365)) #316 what is this??


number of truck cells in row 1096
number of isotope cells in row 2190


...

number of truck cells in row 1096
number of isotope cells in row 2190


<ipython-input-5-3bf58e632af7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed["fraction"] = pd.Series(0, index = range(365)) #316 what is this??


number of truck cells in row 1096
number of isotope cells in row 2190


<ipython-input-5-3bf58e632af7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed["fraction"] = pd.Series(0, index = range(365)) #316 what is this??


number of truck cells in row 1096
number of isotope cells in row 2190


In [13]:
# print(big_df.shape)
# big_df.describe()

In [14]:
#big_df.isnull().sum().sum()

In [15]:
simple = big_df.copy()

## Statistics

In [16]:
simple.fillna(value = 0, axis=1, inplace = True)

simple.head()

,diversion,UraniumMine--enrichment|time0,enrichment--reactor|time0,reactor--SpentFuelSink|time0,UraniumMine--enrichment|time1,enrichment--reactor|time1,reactor--SpentFuelSink|time1,UraniumMine--enrichment|time2,enrichment--reactor|time2,reactor--SpentFuelSink|time2,...,135/131m_t363,133m/133_t363,133m/131m_t363,133/131m_t363,135/133m_t364,135/133_t364,135/131m_t364,133m/133_t364,133m/131m_t364,133/131m_t364
0,False,0.0,0,0,0.0,0,0,0.0,0,0,...,2.879362e+06,0.598893,2072.001477,17234.889314,6895.949083,838.863883,2.900911e+06,0.599359,2074.257106,17240.247335
1,False,0.0,0,0,0.0,0,0,0.0,0,0,...,3.423969e+06,0.717517,2480.934105,20668.120500,8204.926664,997.669148,3.449594e+06,0.718075,2483.634907,20674.545852
2,True,0.0,0,0,0.0,0,0,0.0,0,0,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
3,False,0.0,0,0,0.0,0,0,0.0,0,0,...,2.849183e+06,0.598021,2067.830928,17224.647911,6831.248105,830.324470,2.870507e+06,0.598486,2070.082016,17230.002749
4,False,0.0,0,0,0.0,0,0,0.0,0,0,...,2.879362e+06,0.598893,2072.001477,17234.889314,6895.949083,838.863883,2.900911e+06,0.599359,2074.257106,17240.247335


In [17]:
print(sum(simple["diversion"]))
#simple.describe()

#simple.isnull().sum().sum()

2


In [18]:
from sklearn.model_selection import train_test_split
#split into training and test sets
X = simple.loc[:, simple.columns != "diversion"]
y = simple["diversion"]
trucks_train, trucks_test, diversion_train, diversion_test = train_test_split(X, y, test_size=0.3)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
s_trucks_train, s_trucks_test, diversion_train, diversion_test = train_test_split(X_scaled, y, test_size=0.3)

In [19]:
#random forest classifier
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=8)
rf.fit(s_trucks_train, diversion_train)

rfy_pred = rf.predict(s_trucks_test)

In [20]:
from sklearn import metrics
#check performance
print("Random forest accuracy:",metrics.accuracy_score(diversion_test, rfy_pred))

features = pd.Series(rf.feature_importances_,index=simple.columns[1:]).sort_values(ascending=False)
#print([f for f in features])

Random forest accuracy: 1.0


In [21]:
from sklearn.svm import LinearSVC

linsvc = LinearSVC(C=1)
linsvc.fit(s_trucks_train, diversion_train)
linsvy_pred = linsvc.predict(s_trucks_test)

print("Linear SVC accuracy:", metrics.accuracy_score(diversion_test, linsvy_pred))

from sklearn import svm

svc = svm.SVC()
svc.fit(s_trucks_train, diversion_train)
svc_yhat = svc.predict(s_trucks_test)

print("SVC accuracy:", metrics.accuracy_score(diversion_test, svc_yhat))

Linear SVC accuracy: 1.0
SVC accuracy: 1.0


In [22]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver = "sag")
lrmod = lr.fit(s_trucks_train, diversion_train)
lryhat = lr.predict(s_trucks_test)

print("Logistic regression accuracy:", metrics.accuracy_score(diversion_test, lryhat))

Logistic regression accuracy: 1.0


/Users/Anna/.pyenv/versions/3.8.6/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [23]:
from sklearn import neighbors 
knn = neighbors.KNeighborsClassifier(3, weights = 'uniform')   
model = knn.fit(s_trucks_train, diversion_train)
print(model.score(s_trucks_train, diversion_train))

knnhat = model.predict(s_trucks_test)
print("KNN test accuracy:", metrics.accuracy_score(diversion_test, knnhat))

1.0
KNN test accuracy: 1.0


In [24]:
#what if it's raining? remove 30% of the data points 
X = simple.loc[:, simple.columns != "diversion"]
y = simple["diversion"]

#go through each column, select 30% of rows in that column, change value to 0
def mask(df, proportion):
    new = df.copy()
    for c in range(new.shape[1]):
        row_ids = random.sample(range(new.shape[0]), int(new.shape[0] * proportion))
        for row in row_ids: 
            new.iat[row, c] = 0 
    return new

X30 = mask(X, 0.3)
print(X30.equals(X))


trucks_train, trucks_test, diversion_train, diversion_test = train_test_split(X30, y, test_size=0.3)

scaler = StandardScaler()
X30_scaled = scaler.fit_transform(X30)
sn_trucks_train, sn_trucks_test, diversion_train, diversion_test = train_test_split(X30_scaled, y, test_size=0.3)

False


In [25]:
missing = []
for p in range(1, 8): 
    missing.append(mask(X, p * 0.1))

In [26]:
missing[0].equals(missing[1])

False

In [27]:
scaler = StandardScaler()
train_test = []
scaled = []
for m in missing: 
    train_test.append(train_test_split(m, simple["diversion"], test_size=0.3))
    mscaled = scaler.fit_transform(m)
    scaled.append(train_test_split(mscaled, simple["diversion"], test_size = 0.3))
  
    

In [28]:
X30.head()

,UraniumMine--enrichment|time0,enrichment--reactor|time0,reactor--SpentFuelSink|time0,UraniumMine--enrichment|time1,enrichment--reactor|time1,reactor--SpentFuelSink|time1,UraniumMine--enrichment|time2,enrichment--reactor|time2,reactor--SpentFuelSink|time2,UraniumMine--enrichment|time3,...,135/131m_t363,133m/133_t363,133m/131m_t363,133/131m_t363,135/133m_t364,135/133_t364,135/131m_t364,133m/133_t364,133m/131m_t364,133/131m_t364
0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,...,2.879362e+06,0.000000,2072.001477,17234.889314,6895.949083,0.000000,2.900911e+06,0.000000,2074.257106,0.000000
1,0.0,0,0,0.0,0,0,0.0,0,0,0.0,...,3.423969e+06,0.717517,2480.934105,20668.120500,0.000000,997.669148,0.000000e+00,0.718075,0.000000,20674.545852
2,0.0,0,0,0.0,0,0,0.0,0,0,0.0,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
3,0.0,0,0,0.0,0,0,0.0,0,0,0.0,...,0.000000e+00,0.598021,2067.830928,17224.647911,6831.248105,830.324470,0.000000e+00,0.598486,2070.082016,17230.002749
4,0.0,0,0,0.0,0,0,0.0,0,0,0.0,...,2.879362e+06,0.000000,2072.001477,17234.889314,6895.949083,0.000000,2.900911e+06,0.599359,2074.257106,17240.247335


In [29]:
svc = svm.SVC()
svc.fit(sn_trucks_train, diversion_train)
svc_yhat = svc.predict(sn_trucks_test)

print("SVC accuracy:", metrics.accuracy_score(diversion_test, svc_yhat))

SVC accuracy: 1.0


In [30]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(trucks_train, diversion_train)

rfy_pred = rf.predict(trucks_test)
print("Random forest accuracy:",metrics.accuracy_score(diversion_test, rfy_pred))
print("Random forest auc:",metrics.roc_auc_score(diversion_test, rfy_pred))
print("Random forest confusion matrix:", metrics.confusion_matrix(diversion_test, rfy_pred))

Random forest accuracy: 0.5
Random forest auc: 0.33333333333333337
Random forest confusion matrix: [[2 1]
 [1 0]]


In [31]:
knn = neighbors.KNeighborsClassifier(3, weights = 'uniform')   
model = knn.fit(sn_trucks_train, diversion_train)
print(model.score(sn_trucks_train, diversion_train))

knnhat = model.predict(sn_trucks_test)
print("KNN test accuracy:", metrics.accuracy_score(diversion_test, knnhat))
print("KNN auc:", metrics.roc_auc_score(diversion_test, knnhat))

0.875
KNN test accuracy: 0.75
KNN auc: 0.5


In [32]:
rf_accuracy = []
rf_auc = []
for trucks_train, trucks_test, diversion_train, diversion_test in train_test: 
    rf = RandomForestClassifier(n_estimators=100)
    rf.fit(trucks_train, diversion_train)

    rfy_pred = rf.predict(trucks_test)
    rf_accuracy.append(metrics.accuracy_score(diversion_test, rfy_pred))
    rf_auc.append(metrics.roc_auc_score(diversion_test, rfy_pred))
    print("Random forest accuracy:",metrics.accuracy_score(diversion_test, rfy_pred))
    print("Random forest auc:",metrics.roc_auc_score(diversion_test, rfy_pred))
    print("Random forest confusion matrix:", metrics.confusion_matrix(diversion_test, rfy_pred))
    print()
    
print(rf_accuracy)
print(rf_auc)

Random forest accuracy: 1.0
Random forest auc: 1.0
Random forest confusion matrix: [[3 0]
 [0 1]]

Random forest accuracy: 1.0
Random forest auc: 1.0
Random forest confusion matrix: [[3 0]
 [0 1]]

Random forest accuracy: 1.0
Random forest auc: 1.0
Random forest confusion matrix: [[3 0]
 [0 1]]



ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [ ]:
svc_accuracy = []
svc_auc = []
knn_accuracy = []
knn_auc = []
for s_trucks_train, s_trucks_test, diversion_train, diversion_test in scaled:
    svc = svm.SVC()
    svc.fit(s_trucks_train, diversion_train)
    svc_yhat = svc.predict(s_trucks_test)
    svc_accuracy.append(metrics.accuracy_score(diversion_test, svc_yhat))
    svc_auc.append(metrics.roc_auc_score(diversion_test, svc_yhat))
    print("SVC accuracy:", metrics.accuracy_score(diversion_test, svc_yhat))
    print("SVC auc:",metrics.roc_auc_score(diversion_test, svc_yhat))
    print("SVC confusion matrix:", metrics.confusion_matrix(diversion_test, svc_yhat))
    print()
    
    knn = neighbors.KNeighborsClassifier(3, weights = 'uniform')   
    model = knn.fit(s_trucks_train, diversion_train)
    print(model.score(s_trucks_train, diversion_train))

    knnhat = model.predict(s_trucks_test)
    knn_accuracy.append(metrics.accuracy_score(diversion_test, knnhat))
    knn_auc.append(metrics.roc_auc_score(diversion_test, knnhat))
    print("KNN test accuracy:", metrics.accuracy_score(diversion_test, knnhat))
    print("KNN auc:",metrics.roc_auc_score(diversion_test, knnhat))
    print("KNN confusion matrix:", metrics.confusion_matrix(diversion_test, knnhat))
    print()

In [ ]:
#for anna's model: 

print(svc_accuracy)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(10,8))

#percent of data missing from the set on x axis, accuracy on y axis
plt.scatter([10, 20, 30, 40, 50, 60, 70], rf_accuracy, marker='o', color='b', alpha=0.7,
            s = 124, label='Random Forest Classifier')

plt.scatter([10, 20, 30, 40, 50, 60, 70], svc_accuracy, marker='o', color='r', alpha=0.7, 
            s = 124, label='Support Vector Classifier')

plt.scatter([10, 20, 30, 40, 50, 60, 70], knn_accuracy, marker='o', color='g', alpha=0.7, 
            s = 124, label='K-Nearest Neighbors Classifier')
plt.title("Accuracy of Classifiers for Schema A")
plt.xlabel("Percent of Data Excluded from Model")
plt.ylabel("Accuracy")
plt.xlim(0, 80)
plt.ylim(0.8, 1.05)
plt.legend(loc='lower left')
plt.show()

In [ ]:

plt.figure(figsize=(10,8))

#percent of data missing from the set on x axis, accuracy on y axis
plt.scatter([10, 20, 30, 40, 50, 60, 70], rf_auc, marker='o', color='b', alpha=0.7,
            s = 124, label='Random Forest AUC')

plt.scatter([10, 20, 30, 40, 50, 60, 70], svc_auc, marker='o', color='r', alpha=0.7, 
            s = 124, label='SVC AUC')

plt.scatter([10, 20, 30, 40, 50, 60, 70], knn_auc, marker='o', color='g', alpha=0.7, 
            s = 124, label='KNN AUC')
plt.title("AUC of Classifiers for Schema A")
plt.xlabel("Percent of Data Excluded from Model")
plt.ylabel("AUC")
plt.xlim(0, 80)
plt.ylim(0.8, 1.05)
plt.legend(loc='lower left')
plt.show()

In [ ]:
#for baptiste's model: 
#use big_df
X = big_df.loc[:, big_df.columns != "diversion"]
y = big_df["diversion"]
trucks_train, trucks_test, diversion_train, diversion_test = train_test_split(X, y, test_size=0.3)

missing = []
for p in range(1, 8): 
    missing.append(mask(X, p * 0.1))

In [ ]:
scaler = StandardScaler()
train_test = []
scaled = []
for m in missing: 
    train_test.append(train_test_split(m, big_df["diversion"], test_size=0.3))
    mscaled = scaler.fit_transform(m)
    scaled.append(train_test_split(mscaled, big_df["diversion"], test_size = 0.3))
  

In [ ]:
rf_accuracy = []
rf_auc = []
for trucks_train, trucks_test, diversion_train, diversion_test in train_test: 
    rf = RandomForestClassifier(n_estimators=3)
    rf.fit(trucks_train, diversion_train)

    rfy_pred = rf.predict(trucks_test)
    rf_accuracy.append(metrics.accuracy_score(diversion_test, rfy_pred))
    rf_auc.append(metrics.roc_auc_score(diversion_test, rfy_pred))
    print("Random forest accuracy:",metrics.accuracy_score(diversion_test, rfy_pred))
    print("Random forest auc:",metrics.roc_auc_score(diversion_test, rfy_pred))
    print("Random forest confusion matrix:", metrics.confusion_matrix(diversion_test, rfy_pred))
    print()
    
print(rf_accuracy)
print(rf_auc)

In [ ]:
svc_accuracy = []
svc_auc = []
knn_accuracy = []
knn_auc = []
for s_trucks_train, s_trucks_test, diversion_train, diversion_test in scaled:
    svc = svm.SVC()
    svc.fit(s_trucks_train, diversion_train)
    svc_yhat = svc.predict(s_trucks_test)
    svc_accuracy.append(metrics.accuracy_score(diversion_test, svc_yhat))
    svc_auc.append(metrics.roc_auc_score(diversion_test, svc_yhat))
    print("SVC accuracy:", metrics.accuracy_score(diversion_test, svc_yhat))
    print("SVC auc:",metrics.roc_auc_score(diversion_test, svc_yhat))
    print("SVC confusion matrix:", metrics.confusion_matrix(diversion_test, svc_yhat))
    print()
    
    knn = neighbors.KNeighborsClassifier(3, weights = 'uniform')   
    model = knn.fit(s_trucks_train, diversion_train)
    print(model.score(s_trucks_train, diversion_train))

    knnhat = model.predict(s_trucks_test)
    knn_accuracy.append(metrics.accuracy_score(diversion_test, knnhat))
    knn_auc.append(metrics.roc_auc_score(diversion_test, knnhat))
    print("KNN test accuracy:", metrics.accuracy_score(diversion_test, knnhat))
    print("KNN auc:",metrics.roc_auc_score(diversion_test, knnhat))
    print("KNN confusion matrix:", metrics.confusion_matrix(diversion_test, knnhat))
    print()

In [ ]:
print(knn_accuracy)
print(svc_accuracy)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(10,8))

#percent of data missing from the set on x axis, accuracy on y axis
plt.scatter([10, 20, 30, 40, 50, 60, 70], rf_accuracy, marker='o', color='b', alpha=0.7,
            s = 124, label='Random Forest Classifier')

plt.scatter([10, 20, 30, 40, 50, 60, 70], svc_accuracy, marker='o', color='r', alpha=0.7, 
            s = 124, label='Support Vector Classifier')

plt.scatter([10, 20, 30, 40, 50, 60, 70], knn_accuracy, marker='o', color='g', alpha=0.7, 
            s = 124, label='K-Nearest Neighbors Classifier')
plt.title("Accuracy of Classifiers for Schema B")
plt.xlabel("Percent of Data Excluded from Model")
plt.ylabel("Accuracy")
plt.xlim(0, 80)
plt.ylim(0.75, 1.05)
plt.legend(loc='lower left')
plt.show()

In [ ]:
plt.figure(figsize=(10,8))

#percent of data missing from the set on x axis, accuracy on y axis
plt.scatter([10, 20, 30, 40, 50, 60, 70], rf_auc, marker='o', color='b', alpha=0.7,
            s = 124, label='Random Forest AUC')

plt.scatter([10, 20, 30, 40, 50, 60, 70], svc_auc, marker='o', color='r', alpha=0.7, 
            s = 124, label='SVC AUC')

plt.scatter([10, 20, 30, 40, 50, 60, 70], knn_auc, marker='o', color='g', alpha=0.7, 
            s = 124, label='KNN AUC')
plt.title("AUC of Classifiers for Schema B")
plt.xlabel("Percent of Data Excluded from Model")
plt.ylabel("AUC")
plt.xlim(0, 80)
plt.ylim(0.75, 1.05)
plt.legend(loc='lower left')
plt.show()